In [1]:
using ShipMMG
using JuMP
using Ipopt
using CSV, DataFrames, ProgressMeter, Distributions

In [2]:
# 船体に働く流体力
function X_H(u, v, r)
    U = sqrt((u^2) + ((v - r * x_G)^2))
    if U == 0
        β = 0.0
        r_dash = 0.0
        v_dash = 0.0
    else
        β = asin(-(v - r * x_G) / U)
        r_dash = r * L_pp / U
        v_dash = v / U
    end
    return 0.5 * ρ * L_pp * d * (U^2) * X_H_dash(v_dash, r_dash)
end

function Y_H(u, v, r)
    U = sqrt((u^2) + ((v - r * x_G)^2))
    if U == 0
        β = 0.0
        r_dash = 0.0
        v_dash = 0.0
    else
        β = asin(-(v - r * x_G) / U)
        r_dash = r * L_pp / U
        v_dash = v / U
    end
    return 0.5 * ρ * L_pp * d * (U^2) * Y_H_dash(v_dash, r_dash)
end

function N_H(u, v, r)
    U = sqrt((u^2) + ((v - r * x_G)^2))
    if U == 0
        β = 0.0
        r_dash = 0.0
        v_dash = 0.0
    else
        β = asin(-(v - r * x_G) / U)
        r_dash = r * L_pp / U
        v_dash = v / U
    end
    return 0.5 * ρ * (L_pp^2) * d * (U^2) * N_H_dash(v_dash, r_dash)
end

# 船体に働く流体力(無次元)
function X_H_dash(v_dash, r_dash)
    return -R_0_dash + X_vv_dash*(v_dash)^2 + X_vr_dash*v_dash*r_dash + X_rr_dash*(r_dash^2) + X_vvvv_dash*(v_dash^4)
end

function Y_H_dash(v_dash, r_dash)
    return Y_v_dash*v_dash + Y_r_dash*r_dash + Y_vvr_dash*(v_dash)^2*r_dash + Y_vrr_dash*v_dash*r_dash^2 + Y_vvv_dash*(v_dash^3) + Y_rrr_dash*r_dash^3
end

function N_H_dash(v_dash, r_dash)
    return N_v_dash*v_dash + N_r_dash*r_dash + N_vvr_dash*(v_dash^2)*r_dash + N_vrr_dash*v_dash*r_dash^2 + N_vvv_dash*(v_dash^3) + N_rrr_dash*r_dash^3
end

# 舵に働く力
function X_R(u, v, r, δ, n_p)
    return -(1 - t_R) * F_N(u, v, r, δ, n_p) * sin(δ)
end

function Y_R(u, v, r, δ, n_p)
    return -(1 + a_H) * F_N(u, v, r, δ, n_p) * cos(δ)
end

function N_R(u, v, r, δ, n_p)
    return -(x_R + a_H * x_H) * F_N(u, v, r, δ, n_p) * cos(δ)
end

function F_N(u, v, r, δ, n_p)
    U = sqrt((u^2) + ((v - r * x_G)^2))
    if U == 0
        β = 0.0
        r_dash = 0.0
        v_dash = 0.0
    else
        β = asin(-(v - r * x_G) / U)
        r_dash = r * L_pp / U
        v_dash = v / U
    end
    w_P = w_P0 * exp(-4.0 * (β - x_P * r_dash)^2)
    u_p = (1 - w_P) * u
    if J(u, v, r, n_p) == 0.0
        u_R = sqrt(η * (κ * ϵ * 8.0 * k_0 * n_p^2 * D_p^4 / pi)^2)
    else
        u_R = u * (1.0 - w_P) * ϵ * sqrt(η * (1.0 + κ * (sqrt(1.0 + 8.0 * K_T(u, v, r, n_p) / (pi * J(u, v, r, n_p)^2)) - 1))^2 + (1 - η))
    end
    β_R = β - l_R * r_dash
    if β_R < 0.0
        γ_R = γ_R_minus
    else
        γ_R = γ_R_plus
    end 
    v_R = U * γ_R * β_R
    α_R = δ - atan(v_R, u_R)
    U_R = sqrt(u_R^2 + v_R^2)
    return 0.5 * ρ * A_R * (U_R^2) * f_α * sin(α_R)
end

# プロペラに働く力
function X_P(u, v, r, δ, n_p)
    return (1 - t_P) * T_P(u, v, r, n_p)
end

function K_T(u, v, r, n_p)
    return k_0 + k_1 * J(u, v, r, n_p) + k_2 * (J(u, v, r, n_p)^2)
end

function J(u, v, r, n_p)
    U = sqrt((u^2) + ((v - r * x_G)^2))
    if U == 0
        β = 0.0
        r_dash = 0.0
        v_dash = 0.0
    else
        β = asin(-(v - r * x_G) / U)
        r_dash = r * L_pp / U
        v_dash = v / U
    end
    w_P = w_P0 * exp(-4.0 * (β - x_P * r_dash)^2)
    return u * (1 - w_P) / (n_p * D_p)
end

function T_P(u, v, r, n_p)
    return K_T(u, v, r, n_p) * ρ * n_p^2 * D_p^4
end

T_P (generic function with 1 method)

In [3]:
ρ = 1025.0
L_pp = 7.00  # 船長Lpp[m]
B = 1.27  # 船幅[m]
d = 0.46  # 喫水[m]
nabla = 3.27  # 排水量[m^3]
x_G = 0.25  # 重心位置[m]
# C_b = 0.810  # 方形係数[-]
D_p = 0.216  # プロペラ直径[m]
H_R = 0.345  # 舵高さ[m]
A_R = 0.0539  # 舵断面積[m^2]
t_P = 0.220  # 推力減少率
w_P0 = 0.40  # 有効伴流率
m_x_dash = 0.022  # 付加質量x(無次元)
m_y_dash = 0.223  # 付加質量y(無次元)
J_z_dash = 0.011  # 付加質量Izz(無次元)
t_R = 0.387  # 操縦抵抗減少率
x_R_dash = -0.500  # 舵の相対位置
a_H = 0.312  # 舵力増加係数
x_H_dash = -0.464  # 舵力増分作用位置
γ_R_minus = 0.395  # 整流係数
γ_R_plus = 0.640  # 整流係数
l_r_dash = -0.710  # 船長に対する舵位置
x_P_dash = -0.690  # 船長に対するプロペラ位置
ϵ = 1.09  # プロペラ・舵位置伴流係数比
κ = 0.50  # 修正係数
f_α = 2.747  # 直圧力勾配係数

L_pp = L_pp  # 船長Lpp[m]
B = B  # 船幅[m]
d = d  # 喫水[m]
x_G = x_G  # 重心位置[]
D_p = D_p  # プロペラ直径[m]
m = ρ * nabla  # 質量(無次元化)[kg]
I_zG = ρ * nabla * ((0.25 * L_pp)^2)  # 慣性モーメント[-]
A_R = A_R  # 船の断面に対する舵面積比[-]
η = D_p / H_R  # プロペラ直径に対する舵高さ(Dp/H)
m_x = (0.5 * ρ * (L_pp^2) * d) * m_x_dash  # 付加質量x(無次元)
m_y = (0.5 * ρ * (L_pp^2) * d) * m_y_dash  # 付加質量y(無次元)
J_z = (0.5 * ρ * (L_pp^4) * d) * J_z_dash  # 付加質量Izz(無次元)
f_α = f_α # 直圧力勾配係数
ϵ = ϵ  # プロペラ・舵位置伴流係数比
t_R = t_R  # 操縦抵抗減少率
x_R = x_R_dash * L_pp  # 舵の位置
a_H = a_H  # 舵力増加係数
x_H = x_H_dash * L_pp  # 舵力増分作用位置
γ_R_minus = γ_R_minus  # 整流係数
γ_R_plus = γ_R_plus  # 整流係数
l_R = l_r_dash  # 船長に対する舵位置
κ = κ  # 修正係数
t_P = t_P  # 推力減少率
w_P0 = w_P0  # 有効伴流率
x_P = x_P_dash  # 船長に対するプロペラ位置

const k_0 = 0.2931
const k_1 = -0.2753
const k_2 = -0.1385

L_f = 3.0 # センサー位置(船首)
L_t = 3.0 # センサー位置(船尾)

3.0

In [4]:
p = [
    [0.0200, -0.0799, 0.1016, -0.0176, 0.5704, -0.3711, 0.0720, -1.0535, 0.2070, -0.2556, 0.0220, -0.1097, -0.0527, -0.0533, -0.2863, -0.0206, -0.0196],
    [0.022, -0.040, 0.002, 0.011, 0.771, -0.315, 0.083, -1.607, 0.379, -0.391, 0.008, -0.137, -0.049, -0.030, -0.294, 0.055, -0.013]
] 

q=2
R_0_dash = p[q][1]
X_vv_dash = p[q][2]
X_vr_dash = p[q][3]
X_rr_dash = p[q][4]
X_vvvv_dash = p[q][5]
Y_v_dash = p[q][6]
Y_r_dash = p[q][7]
Y_vvv_dash = p[q][8]
Y_vvr_dash = p[q][9]
Y_vrr_dash = p[q][10]
Y_rrr_dash = p[q][11]
N_v_dash = p[q][12]
N_r_dash = p[q][13]
N_vvv_dash = p[q][14]
N_vvr_dash = p[q][15]
N_vrr_dash = p[q][16]
N_rrr_dash = p[q][17]

-0.013

In [5]:
function mpc(f_x_t::Vector{Float64}, δ_ref, n_p_ref, f_obs::DataFrame ,t::Int, N::Int)
    global R_0_dash, X_vv_dash, X_vr_dash, X_rr_dash, X_vvvv_dash
    global Y_v_dash, Y_r_dash, Y_vvv_dash, Y_vvr_dash, Y_vrr_dash, Y_rrr_dash
    global N_v_dash, N_r_dash, N_vvv_dash, N_vvr_dash, N_vrr_dash, N_rrr_dash

    mcmc = CSV.read("../MCMC/result/10degL2MCMC_KVLCC2.csv", DataFrame)
    # モデル作成
    model = Model(Ipopt.Optimizer) # Ipoptを使用
    # set_time_limit_sec(model, 6.0)
    set_silent(model) # 計算の進行状況を非表示

    Q = 1*10^8 # 座標誤差への重み
    R1 = 1.0 # 制御量変化(X_F)への重み
    R2 = 1.0# 制御量変化(Y_F)への重み
    R3 = 1.0# 制御量変化(N_F)への重み

    # 方程式を登録
    register(model, :X_H, 3, X_H; autodiff = true)
    register(model, :Y_H, 3, Y_H; autodiff = true)
    register(model, :N_H, 3, N_H; autodiff = true)
    register(model, :X_H_dash, 2, X_H_dash; autodiff = true)
    register(model, :Y_H_dash, 2, Y_H_dash; autodiff = true)
    register(model, :N_H_dash, 2, N_H_dash; autodiff = true)
    register(model, :X_R, 5, X_R; autodiff = true)
    register(model, :Y_R, 5, Y_R; autodiff = true)
    register(model, :N_R, 5, N_R; autodiff = true)
    register(model, :F_N, 5, F_N; autodiff = true)
    register(model, :X_P, 5, X_P; autodiff = true)
    register(model, :K_T, 4, K_T; autodiff = true)
    register(model, :J, 4, J; autodiff = true)
    register(model, :T_P, 4, T_P; autodiff = true)
    
    # 使用する変数を設定
    x = @variable(model, [t:t+N+2, 1:13]) # 操作量 x
    u1 = @variable(model, [t:t+N+1, 1:1]) # 制御量 u1
    u2 = @variable(model, [t:t+N+1, 1:1]) # 制御量 u2
    u3 = @variable(model, [t:t+N+1, 1:1])

    # 制御量の制約条件
    @constraint(model, -75.0 .<= u1[t:t+N+1, 1:1] .<= 75.0) 
    @constraint(model, -200.0 .<= u2[t:t+N+1, 1:1] .<= 200.0)
    @constraint(model, -150.0 .<= u3[t:t+N+1, 1:1] .<= 150.0)
    
    # 操作量の箱を作成
    for k in t:t+N+2
        set_start_value.(x[k, 1], f_x_t[1])
        set_start_value.(x[k, 2], f_x_t[2])
        set_start_value.(x[k, 3], f_x_t[3])
        set_start_value.(x[k, 4], f_x_t[4])
        set_start_value.(x[k, 5], f_x_t[5])
        set_start_value.(x[k, 6], f_x_t[6])
        set_start_value.(x[k, 7], f_x_t[7])
        set_start_value.(x[k, 8], f_x_t[8])
        set_start_value.(x[k, 9], f_x_t[9])
        set_start_value.(x[k, 10], f_x_t[10])  
    end
    
    # 初期状態(t)での変数xの設定
    for i in 1:10
        @constraint(model, x[t, i] == f_x_t[i])
    end

    # MMGを用いた制約
    for k in t:t+N+1
        random_num = rand(1:1000)
        R_0_dash = mcmc[random_num,4]
        X_vv_dash = mcmc[random_num,5]
        X_vr_dash = mcmc[random_num,6]
        X_rr_dash = mcmc[random_num,7]
        X_vvvv_dash = mcmc[random_num,8]
        Y_v_dash = mcmc[random_num,9]
        Y_r_dash = mcmc[random_num,10]
        Y_vvv_dash = mcmc[random_num,11]
        Y_vvr_dash = mcmc[random_num,12]
        Y_vrr_dash = mcmc[random_num,13]
        Y_rrr_dash = mcmc[random_num,14]
        N_v_dash = mcmc[random_num,15]
        N_r_dash = mcmc[random_num,16]
        N_vvv_dash = mcmc[random_num,17]
        N_vvr_dash = mcmc[random_num,18]
        N_vrr_dash = mcmc[random_num,19]
        N_rrr_dash = mcmc[random_num,20]
        @NLconstraint(model, x[k+1,1] == x[k,1] + dt * ((X_H(x[k,1], x[k,2], x[k,3]) + X_R(x[k,1], x[k,2], x[k,3], δ_ref[k], n_p_ref[k]) + X_P(x[k,1], x[k,2], x[k,3], δ_ref[k], n_p_ref[k]) + u1[k,1] + (m + m_y) * x[k,2] * x[k,3]) / (m + m_x)))
        @NLconstraint(model, x[k+1,2] == x[k,2] + dt * ((Y_H(x[k,1], x[k,2], x[k,3]) + Y_R(x[k,1], x[k,2], x[k,3], δ_ref[k], n_p_ref[k]) + u2[k,1] - (m + m_x) * x[k,1] * x[k,3]) / (m + m_y)))
        @NLconstraint(model, x[k+1,3] == x[k,3] + dt * ((N_H(x[k,1], x[k,2], x[k,3]) + N_R(x[k,1], x[k,2], x[k,3], δ_ref[k], n_p_ref[k]) + u3[k,1]) / (I_zG + J_z)))
        @NLconstraint(model, x[k+1,4] == x[k,4] + dt * (x[k,1] * cos(x[k,6]) - x[k,2] * sin(x[k,6])))
        @NLconstraint(model, x[k+1,5] == x[k,5] + dt * (x[k,1] * sin(x[k,6]) + x[k,2] * cos(x[k,6])))
        @NLconstraint(model, x[k+1,6] == x[k,6] + dt * (x[k,3]))
        @NLconstraint(model, x[k+1,7] == x[k+1,4] + L_f * cos(x[k+1,6]))
        @NLconstraint(model, x[k+1,8] == x[k+1,5] + L_f * sin(x[k+1,6]))
        @NLconstraint(model, x[k+1,9] == x[k+1,4] + L_t * cos(x[k+1,6] + pi))
        @NLconstraint(model, x[k+1,10] == x[k+1,5] + L_t * sin(x[k+1,6] + pi))
    end

    #評価関数の設定
    @NLobjective(
    model,
    Min,
    sum(Q * ((x[k,7] - f_obs[k,8])^2 + (x[k,8] - f_obs[k,9])^2 + (x[k,9] - f_obs[k,10])^2 + (x[k,10] - f_obs[k,11])^2) for k in t+2:t+N+2)
    + sum(R1 * (u1[k,1] - u1[k-1,1])^2 for k in t+1:t+N)
    + sum(R2 * (u2[k,1] - u2[k-1,1])^2 for k in t+1:t+N)
    + sum(R3 * (u3[k,1] - u3[k-1,1])^2 for k in t+1:t+N))

    # 最適化の実行
    optimize!(model)
    
    # 次点の操作量x(t+1)とそれに用いた制御量u(t)の値を返す
    return value.(x[t+2,1:10]), value.(u1[t,1]), value.(u2[t,1]), value.(u3[t,1]) # 最適化後の入力x・制御量u
end

mpc (generic function with 1 method)

In [6]:
function run_mpc(duration::Int, dt::Float64, q::Int)
    #観測データ
    obs = CSV.read("./observation/obs_35deg_10ms_EF.csv", DataFrame)
    δ_list = obs[!,"δ"]
    n_p_list = obs[!,"n_p"]

    u1_0 = 0.0
    u2_0 = 0.0
    u3_0 = 0.0

    # データフレーム
    cols = Int(duration/dt) + 1
    df_x = DataFrame(
        time = collect(0:dt:duration),
        u = zeros(cols),
        v = zeros(cols),
        r = zeros(cols),
        x = zeros(cols),
        y = zeros(cols),
        ψ = zeros(cols),
        x1 = zeros(cols),
        y1 = zeros(cols),
        x2 = zeros(cols),
        y2 = zeros(cols),
        δ = zeros(cols),
        n_p = zeros(cols),
        X_F = zeros(cols),
        Y_F = zeros(cols),
        N_F = zeros(cols)
    )

    #初期値をデータフレームへ
    df_x[1, 2:13] .= obs[1, 2:13] |> collect
    df_x[2, 2:13] .= obs[2, 2:13] |> collect    

    # MPCの実行
    for t in 1:Int(duration/dt)-1
        x_0 = Vector(Float64[obs[t,"u"], obs[t,"v"], obs[t,"r"], obs[t,"x"], obs[t,"y"], obs[t,"ψ"], obs[t,"x1"], obs[t,"y1"], obs[t,"x2"], obs[t,"y2"]])
        #MPCの結果
        x_val, u1_val, u2_val, u3_val = mpc(x_0, δ_list, n_p_list, obs, t, N)

        #結果をデータフレームへ
        df_x[t+2, 2:11] .= x_val
        df_x[t+2, 12:13] .= [δ_list[t+2], n_p_list[t+2]]
        df_x[t, 14:16] .= [u1_val, u2_val, u3_val]
        
        u1_0 = u1_val
        u2_0 = u2_val
        u3_0 = u3_val

    end

    # 結果書き出し
    CSV.write("./result/10to35/MPC_result$(q).csv", df_x)
end

run_mpc (generic function with 1 method)

In [7]:
N = 16 # 予測ホライゾン
duration = 50 # 全体の計算時間
dt = 0.25 # 刻み幅

@showprogress for q in 1:100
    run_mpc(duration, dt, q)
end


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



Progress: 100%|█████████████████████████████████████████| Time: 6:10:34
